# Project 1 - Airline AI Assistant

For this first project, we'll work on one of the very common use cases for Gen AI: a chatbot

In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import display
import gradio as gr

In [17]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "sydney": "$2999"}

In [ ]:
# Load environment variables in a file called .env

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    

In [3]:
# For our chatbot we will select GPT-4.1-nano due to the price efficiency

MODEL = "gpt-4.1-mini"
openai = OpenAI()

## Our first Assistant conversation

We can call the API with a conversation - a list of dictionaries representing each interaction:

```
[
    {"role": "system", "content": "essential instructions here, including tone"},
    {"role": "user", "content": "a question from the user"},
    {"role": "assistant": "content": "a response"},
    ...
]
```

In [31]:
#system_message = "You are a helpful assistant for an Airline called FlightAI. "
#system_message += "Give short, witty, humorous answers, no more than 1 sentence. "
#system_message += "Give sarcastic answers, no more than 1 sentence. "
#system_message += "Make the answers funny but also consise"
#system_message += "Always be accurate. If you don't know the answer, say so."
system_message = f"""You are helpful assistant for an Airline called FlightAI..
FlightAI operates flights to a few cities with prices stored in: {ticket_prices}

Be helpful and accurate in providing guidance. If you don't know the answer, say so."""

In [ ]:
print(system_message)
ticket_prices

And now we wrap this in a simple chat() function,  
and then we use the shockingly simple Gradio platform to bring up a Chatbot UI.

First, the chat function, which takes the current message and the history of prior messages:

In [33]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

In [ ]:
# And look how complicated it is to launch a User Interface, with the fabulous gradio platform

gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True)

## Expertise

We can give our model knowledge using the system prompt

In [35]:
system_message += " In case it's relevant, the price of a return ticket to London is $799."

In [ ]:
system_message

In [ ]:

gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True)

In [38]:
concerts = [
    {"artist": "Taylor Swift", "city": "New York", "price": "$250"},
    {"artist": "Taylor Swift", "city": "London", "price": "$220"},
    {"artist": "Taylor Swift", "city": "Tokyo", "price": "$280"},

    {"artist": "BTS", "city": "Seoul", "price": "$180"},
    {"artist": "BTS", "city": "Los Angeles", "price": "$210"},
    {"artist": "BTS", "city": "Singapore", "price": "$200"},

    {"artist": "Beyoncé", "city": "Paris", "price": "$230"},
    {"artist": "Beyoncé", "city": "Sydney", "price": "$240"},
    {"artist": "Beyoncé", "city": "Toronto", "price": "$225"},

    {"artist": "Drake", "city": "Toronto", "price": "$210"},
    {"artist": "Drake", "city": "Chicago", "price": "$195"},
    {"artist": "Drake", "city": "Miami", "price": "$200"},

    {"artist": "Ed Sheeran", "city": "Dublin", "price": "$190"},
    {"artist": "Ed Sheeran", "city": "Melbourne", "price": "$205"},
    {"artist": "Ed Sheeran", "city": "Amsterdam", "price": "$215"},

    {"artist": "Adele", "city": "Las Vegas", "price": "$350"},
    {"artist": "Adele", "city": "London", "price": "$330"},
    {"artist": "Adele", "city": "Dubai", "price": "$400"},

    {"artist": "Coldplay", "city": "Berlin", "price": "$180"},
    {"artist": "Coldplay", "city": "Barcelona", "price": "$175"},
    {"artist": "Coldplay", "city": "Mumbai", "price": "$160"},

    {"artist": "The Weeknd", "city": "Los Angeles", "price": "$240"},
    {"artist": "The Weeknd", "city": "Mexico City", "price": "$200"},
    {"artist": "The Weeknd", "city": "Rome", "price": "$220"},

    {"artist": "Billie Eilish", "city": "San Francisco", "price": "$190"},
    {"artist": "Billie Eilish", "city": "London", "price": "$200"},
    {"artist": "Billie Eilish", "city": "Oslo", "price": "$210"},

    {"artist": "Harry Styles", "city": "Manchester", "price": "$230"},
    {"artist": "Harry Styles", "city": "New York", "price": "$260"},
    {"artist": "Harry Styles", "city": "Auckland", "price": "$250"}
]


In [41]:


system_message = f"""You are helpful assistant for a ticketing company called TicketAI.
You get queries from customers looking to find where their favourite artists are playing.
This information is stored in : {concerts}

Be helpful and accurate in providing guidance but do so in a manner that incorporates one of those artists song names. If you don't know the answer, say so."""

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True)

## First steps towards agentic workflows

Having multiple AIs collaborate to solve the problem is a simple step. Let's add another AI to the mix!

Let's pick a fun one: adding multi-modality..

In [43]:
import base64
from io import BytesIO
from PIL import Image

def artist_agent(city):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [ ]:
image = artist_agent("New York")
display(image)

## Bringing it all together in a mini Agent Framework

Bringing together both our LLM calls in 1 UI

In [45]:
def chat(history):
    message = history[-1]["content"]
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    image =  artist_agent("London") if 'london' in message.lower() else None
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]
    return history, image

In [ ]:
# More involved Gradio code as we're not using the preset Chat interface!

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=400, type="messages")
        image_output = gr.Image(height=400)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")

    def do_entry(message, history):
        history += [{"role": "user", "content": message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )

ui.launch(inbrowser=True)

## Congratulations!

You just created a multi-modal AI Assistant

### Simple Assignment

Based on this code:

```python
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "sydney": "$2999"}
```

Allow the Chatbot to quote ticket prices for any of those destinations, and show images!

## MAJOR ASSIGNMENT

Make a variation of this Chatbot that applies this to your business


### Optional Stretch Assignment for office hours

Research "tool use" (also known as Function Calling) and then use this technique to add Tool capabilities to your LLM to look up ticket prices..


In [ ]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "sydney": "$2999"}


In [55]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "sydney": "$2999"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [56]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this when you need to know a ticket price.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False,
    }
}

In [57]:
tools = [{"type": "function", "function": price_function}]

In [59]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [60]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": message.tool_calls[0].id
    }
    return response, city

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True)